In [ ]:
import numpy as np
import pandas as pd
import time
import os

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly_express as px
import plotly.graph_objects as go

import seaborn as sns

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Train data
train = pd.read_csv('/kaggle/input/mercari-price-prediction-train-and-test-data/train.tsv', sep='\t')
test = pd.read_csv('/kaggle/input/mercari-price-prediction-train-and-test-data/test.tsv', sep='\t')

display(train.head())
print('Train data:', train.shape)
#display(train.shape)

print('Test data shape: ', test.shape)

In [ ]:
train.columns

In [ ]:
train.info()

### Check for missing values ###

In [ ]:
train.isnull().sum()[train.isnull().sum().values > 0]

### Target variable - 'price'

In [ ]:
train.price.describe()

### Target variable - 'price' - distribution
* Log transformation to change to normal distribution

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

ax1.hist(train['price'], range=[0,250], bins=50, edgecolor='w')
ax1.set_xlabel('price')
ax1.set_ylabel('distribution')
ax1.set_title('histogram of price')

ax2.hist(np.log1p(train['price']), range=[0,10], bins=30, edgecolor='w')
ax2.set_xlabel('Log of price')
ax2.set_ylabel('distribution')
ax2.set_title('histogram of Log price')
plt.show()

In [ ]:
train['logprice'] = np.log1p(train['price'])
train.head(5)

### * Shipping variable *
- 'shipping':
    - 1 if shipping fee is paid by seller
    - 0 if shipping fee is paid by buyer
    
- 'Shipping fee' is quite evenly split  between the buyer and seller
    - 55% of shipping fee is paid by the buyer

In [ ]:
train['shipping'].value_counts(normalize=True)

### Check for any relationship between shipping fee and price
- the price of good is slightly lesser when the seller pays for the shipping to make the price competitive

In [ ]:
plt.figure(figsize=(15,5))
train.groupby('shipping')['price'].plot(kind='hist', range=[0,100], bins=25, alpha=0.7)
plt.legend(labels=['buyer', 'seller'])
plt.xlabel('price')
plt.ylabel('distribution')
plt.title('variations across buyer and seller')
plt.show()

### * Category types *

In [ ]:
print('Number of unique categories is: {}\n'.format(len(train['category_name'].value_counts())))
print('Top 10 categories: \n{}'.format(train['category_name'].value_counts()[:10]))

In [ ]:
# split the categories

# text = 'Women/Athletic Apparel/Pants, Tights, Leggings'
# text.split('/')

def split_categories(text):
    try:
        return text.split('/')
    # if no category name
    except: 
        return ['no label', 'no label', 'no label']

In [ ]:
train['general_cat'] = train['category_name'].apply(lambda x: split_categories(x)[0])
train['sub_cat1'] = train['category_name'].apply(lambda x: split_categories(x)[1])
train['sub_cat2'] = train['category_name'].apply(lambda x: split_categories(x)[2])

train.head(10)

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(x = train['general_cat'].value_counts().index.values, height = train['general_cat'].value_counts().values)
plt.xlabel('General categories')
plt.ylabel('Count')
plt.show()

In [ ]:
# show in plotly

trace = go.Bar(x = train['general_cat'].value_counts().index.values, 
               y = train['general_cat'].value_counts().values,
                text = round(train['general_cat'].value_counts(normalize=True)*100,2)
              )
layout = go.Layout(dict(
                        title = 'Number of items by general category'),
                        xaxis= dict(title = 'general categories'),
                        yaxis= dict(title = 'Count')
                  )
fig  = go.Figure(data =trace, layout = layout)
py.iplot(fig)

### Sub category 1 - Top 20

In [ ]:
# show in plotly

trace = go.Bar(x = train['sub_cat1'].value_counts().index.values[:20], 
               y = train['sub_cat1'].value_counts().values[:20],
                text = round(train['sub_cat1'].value_counts(normalize=True)*100,2)
              )
layout = go.Layout(dict(
                        title = 'Number of items by sub_category 1'),
                        xaxis= dict(title = 'sub_category 1'),
                        yaxis= dict(title = 'Count')
                  )
fig  = go.Figure(data =trace, layout = layout)
py.iplot(fig)

In [ ]:
# show in plotly

trace = go.Bar(x = train['sub_cat2'].value_counts().index.values[:20], 
               y = train['sub_cat2'].value_counts().values[:20],
                text = round(train['sub_cat2'].value_counts(normalize=True)*100,2),
                             )
layout = go.Layout(dict(
                        title = 'Number of items by sub_category 2'),
                        xaxis= dict(title = 'sub_category 2'),
                        yaxis= dict(title = 'Count'),
                        )
fig  = go.Figure(data =trace, layout = layout)
py.iplot(fig)

In [ ]:
gen_cat = train['general_cat'].unique()
x = [train.loc[train['general_cat'] == cat, 'price'] for cat in gen_cat]

trace = [go.Box(x = np.log1p(x[i]), name = gen_cat[i]) for i in range(len(gen_cat))]
layout = dict(
            title = 'Price distrbution across general category',
            xaxis = dict(title='distribution'),
            yaxis = dict(title = 'category')
            )
fig = go.Figure(data = trace, layout=layout)
py.iplot(fig)

In [ ]:
train.isnull().sum()[train.isnull().sum().values > 0]

In [ ]:
# fill the columns that have missing values

def handle_missing_values(df):
    df['category_name'].fillna(value = 'missing', inplace=True)
    df['brand_name'].fillna(value = 'missing', inplace=True)
    df['item_description'].replace('No description yet', 'missing', inplace=True)
    df['item_description'].fillna(value = 'missing', inplace=True)

### **Brand names**

In [ ]:
print('Number of unique brands is : ', len(train['brand_name'].value_counts()))
print('Number of unique brands with count > 1 : ', len(train['brand_name'].value_counts()[train['brand_name'].value_counts() > 1]))

### Item description
- the data is very unstructured
- has punctuations, numbers etc which need to be removed
- remove stop words
- also remove any words with len < 3

In [ ]:
train['item_description']

In [ ]:
text = train.iloc[1482534]['item_description']
text

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
import re

def tokenize_text(text):
    regex = re.compile(r'[a-zA-Z]{3,}')
    txt = regex.findall(str(text).lower())
    #tokens = [t for t in txt if t not in stop and len(t)>3]
    tokens = [t for t in txt if t not in stop and len(t)>3]
    return tokens

### get count of tokens

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# #cv = CountVectorizer(stop_words='english', token_pattern= r'\b[^\d\W]+\b')
# cv = CountVectorizer(stop_words='english', token_pattern= r'[a-zA-Z]{3,}')
# #cv = CountVectorizer()

# def word_count(text):
#     try:
#         cv.fit([text]) ## if this doesnt work, try 'text' instead of '[text]'
#         return len(cv.get_feature_names())
#     except:
#         return 0

### get tokens

In [ ]:
#cv = CountVectorizer(stop_words='english', token_pattern= r'\b[^\d\W]+\b')
# cv = CountVectorizer(stop_words='english', token_pattern= r'[a-zA-Z]{3,}')
# #cv = CountVectorizer(stop_words='english')

# def word_token(text):
#     try:
#         cv.fit([text]) ## if this doesnt work, try 'text' instead of '[text]'
#         tokens = cv.get_feature_names()
#         return tokens
#     except:
#         return 0

### Looking at a sample of the full dataset

In [ ]:
# trace = go.Scatter(x = df['desc_length'], y = df['price'],
#                   mode = 'lines+markers')
# layout = go.Layout(dict(title='price vz description'))

# fig = go.Figure(data=[trace], layout=layout)
# fig.show()

### Taking the complete train dataset

### test a sample

In [ ]:
train_sample = train.sample(frac=0.001, random_state=123)
train_sample['tokens'] = train_sample['item_description'].apply(lambda x: tokenize_text(x))
train_sample['desc_length'] = train_sample['tokens'].apply(lambda x: len(x))

train_sample.head(10)

for description, tokens, length in zip(train_sample['item_description'], train_sample['tokens'], train_sample['desc_length']):
    print('Item description: ', description)
    print('Word tokens', tokens)
    print('Description length: ', length)
    print('\n')

### Taking the complete train dataset

In [ ]:
# train['tokens'] = train['item_description'].apply(lambda x: tokenize_text(x))
# train['desc_length'] = train['tokens'].apply(lambda x: len(x))

# train.head(10)

# for description, tokens, length in zip(train_sample['item_description'], train_sample['tokens'], train_sample['desc_length']):
#     print('Item description: ', description)
#     print('Word tokens', tokens)
#     print('Description length: ', length)
#     print('\n')

In [ ]:
# train_sample = train.sample(frac=0.001, random_state=10)
# train_sample['tokens'] = train_sample['item_description'].apply(lambda x: tokenize_text([x]))
# train_sample['desc_length'] = train_sample['item_description'].apply(lambda x: word_count([x]))
# train_sample[['tokens','desc_length']].head(10)

In [ ]:
import datetime
import time

start = time.time()
print('Start time:', datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
train['tokens'] = train['item_description'].apply(lambda x: tokenize_text(x))
train['desc_length'] = train['tokens'].apply(lambda x: len(x))

print('time taken for train:', time.time() - start)
print('End time:', datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))

# for test
start = time.time()
print('Start time:', datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
test['tokens'] = test['item_description'].apply(lambda x: tokenize_text(x))
test['desc_length'] = test['tokens'].apply(lambda x: len(x))
print('time taken for test:', time.time() - start)
# print('End time:', datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))


In [ ]:
train.head(10)

### Saving the file into a new csv file

In [ ]:
train.to_csv('train_desc.csv',index=False)
test.to_csv('test_desc.csv',index=False)

In [ ]:
# train = pd.read_csv('/kaggle/input/mercari-price-suggestion-with-desc/train_desc.csv')
# test = pd.read_csv('/kaggle/input/mercari-price-suggestion-with-desc/test_desc.csv')
# train.head()

### * Mean price for description length *

In [ ]:
df = train.groupby('desc_length')['price'].mean().reset_index()
df

In [ ]:
trace = go.Scatter(
                    x = df['desc_length'],
                    y = df['price'],
                    mode = 'lines+markers',
                    )

layout = go.Layout(dict(title = 'Average price per description length',
                       xaxis = dict(title = 'Description length'),
                       yaxis = dict(title = 'price')
                       )
                  )
fig = go.Figure(data = [trace], layout=layout)
py.iplot(fig)

In [ ]:
display(train.isnull().sum())
display(train.shape)

In [ ]:
# removing the entries which do not have a 'item description'

train_new = train[pd.notnull(train['item_description'])]
display(train_new.isnull().sum())
display(train_new.shape)

### General categories

In [ ]:
train['general_cat'].value_counts()

### Create a 'WordCloud' for the most common words in each 'general' category

In [ ]:
from wordcloud import WordCloud
from collections import Counter

cat_desc = dict()
for cat in train_sample['general_cat'].value_counts().index.tolist():
    text = " ".join(train_sample.loc[train_sample['general_cat'] == cat, 'item_description'].values)
    cat_desc[cat]= tokenize_text(text)

womens100 = Counter(cat_desc['Women']).most_common(100)
beauty100 = Counter(cat_desc['Beauty']).most_common(100)
kids100 = Counter(cat_desc['Kids']).most_common(100)
electronics100 = Counter(cat_desc['Electronics']).most_common(100)

In [ ]:
keys = [k for (k,v) in womens100]
text = " ".join(keys)
text

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#optional - to remove the word 'shipping' from the wordcloud
stopwords = set(STOPWORDS)
#print(stopwords)
stopwords.update(['shipping', 'Shipping', "shipping'"])
#optional

def create_wordcloud(text):
    wordcloud = WordCloud(background_color='white', stopwords=stopwords,
                          max_font_size=30).generate(text)
    return wordcloud

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(15, 10))

# wordcloud requires input text to be a string

ax[0,0].imshow(create_wordcloud(str(womens100)), interpolation='bilinear')
ax[0,0].set_title('women', fontsize=25)

ax[0,1].imshow(create_wordcloud(str(beauty100)), interpolation='bilinear')
ax[0,1].set_title('beauty', fontsize=25)

ax[1,0].imshow(create_wordcloud(str(kids100)), interpolation='bilinear')
ax[1,0].set_title('kids', fontsize=25)

ax[1,1].imshow(create_wordcloud(str(electronics100)), interpolation='bilinear')
ax[1,1].set_title('Electronics', fontsize=25)

plt.tight_layout(pad =1)
plt.show()

In [ ]:
# Some random tests

import operator
temp = Counter(cat_desc['Women'])
#sorted(temp.items(), key=operator.itemgetter(1), reverse=True)

lofwords=[]
for k, v  in temp.items():
    if v > 10:
        lofwords.append(k)

len(lofwords)

### * Using Tf-Idf vectorizer *
- It depends on two factors:
    * Terms frequency
    * Inverse Document frequency

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10,\
                             max_features= 180000, \
                             tokenizer = tokenize_text, \
                             ngram_range=(1,2)
                            )

In [ ]:
train_new.isnull().sum()

In [ ]:
# Apply to train and test together
# use train_new, which has no item description missing

start = time.time()
all_desc = np.append(train_new['item_description'].values, test['item_description'].values)
vz = vectorizer.fit_transform(all_desc)
print('Time taken:', time.time() - start)

In [ ]:
# saving the model file
import pickle
pickle.dump(vz, open('tfidf_vectorizer_combined_whole.pkl', 'wb'))

In [ ]:
# extract features
# zip the features and the correspondinf IDF scores together
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

#create dataframe
tfidf = pd.DataFrame(tfidf.items(), columns=['features', 'idf_score'])

### *Top 10 features with the highest tfidf scores *

In [ ]:
tfidf.sort_values('idf_score', ascending=False).head(10)

* Each document is represented as a row vector of shape 180,000

In [ ]:
print('Shape of the vectorized implementation of item description: ',vz.shape)
print('Dimension of the Tf-Idf dataframe is :', tfidf.shape)

### Visualize the data using t-SNE (t-Distributed Stochastic Neighbor Embedding)
* combine train and test data
* sample a fraction from the combined

In [ ]:
train_copy = train_new.copy()
test_copy = test.copy()

train_copy['is_train'] = 1
test_copy['is_train'] = 0


combined_df = pd.concat([train_copy, test_copy], sort=False)

sample_size= 15000
combined_sample = combined_df.sample(n=sample_size, random_state=1)

# applying tfidf vectorizer
#vz_sample = vectorizer.fit_transform(combined_sample['item_description'].values)
vz_sample = vectorizer.transform(combined_sample['item_description'].values)

print('Shape of the sample tfidf matrix is: ', vz_sample.shape)

### Using Truncated SVD to reduce dimensions

In [ ]:
from sklearn.decomposition import TruncatedSVD

n_comp= 30
svd = TruncatedSVD(n_components=n_comp, random_state=1)
svd_tfidf = svd.fit_transform(vz_sample)
print('Shape of svd_tfidf matrix is:', svd_tfidf.shape)

### Use TSNE

In [ ]:
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=42, n_iter=500)

In [ ]:
tsne_tfidf = tsne_model.fit_transform(svd_tfidf)
print('Shape of tsne_tfidf matrix is:', tsne_tfidf.shape)

In [ ]:
# saving the tsne model
pickle.dump(tsne_tfidf, open('tsne_svd_tfidf_combined_sample.pkl', 'wb'))

In [ ]:
combined_sample.head()

In [ ]:
combined_sample.reset_index(drop=True, inplace=True)

In [ ]:
combined_sample.head()

In [ ]:
# After SVD and T-SNE on a sample of the combined train and test datasets

tfidf_df = pd.DataFrame(tsne_tfidf, columns=['pc1', 'pc2'])
tfidf_df['item_description'] = combined_sample['item_description']
tfidf_df['tokens'] = combined_sample['tokens']
tfidf_df['category'] = combined_sample['general_cat']

tfidf_df.head()

In [ ]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook

In [ ]:
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600,
                       title="tf-idf clustering of the item description",
    tools="pan,wheel_zoom,box_zoom,reset,hover, save",
    x_axis_type=None, y_axis_type=None, min_border=1)

In [ ]:
plot_tfidf.scatter(x='pc1', y='pc2', source=tfidf_df, alpha=0.7)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"description": "@item_description", "tokens": "@tokens", "category":"@category"}
show(plot_tfidf)

In [ ]:
#plt.scatter(tfidf_df['pc1'], tfidf_df['pc2'])

### K-Means clustering

In [ ]:
from sklearn.cluster import MiniBatchKMeans

num_clusters=30

kmeans_model = MiniBatchKMeans(n_clusters = num_clusters, 
                               init='k-means++',
                               n_init=1,
                               init_size=1000, 
                               batch_size=1000, 
                               max_iter=1000,
                               verbose=1
                              )

In [ ]:
# Using the original tfidf vectorizer which will be fit on the combined "whole" train and test
vectorizer = TfidfVectorizer(min_df=10,\
                             max_features= 180000, \
                             tokenizer = tokenize_text, \
                             ngram_range=(1,2)
                            )

all_desc = np.append(train_new['item_description'].values, test['item_description'].values)
vz = vectorizer.fit_transform(all_desc)

In [ ]:
print('Number of features or tokens ', len(vectorizer.get_feature_names()))

In [ ]:
# Applying the kmeans clustering to the tfidf matrix, to reduce dimensions
kmeans = kmeans_model.fit(vz)
kmeans_clusters = kmeans_model.predict(vz)
kmeans_distances = kmeans_model.transform(vz)

In [ ]:
# saving the model
pickle.dump(kmeans_model, open('kmeans_tfidf_combined_whole.pkl', 'wb'))

In [ ]:
np.unique(kmeans_clusters, return_counts=True)

In [ ]:
print(kmeans.cluster_centers_.shape)
terms = vectorizer.get_feature_names()

In [ ]:
len(terms)

In [ ]:
sorted_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
sorted_centroids

In [ ]:
for i in range(num_clusters):
    print('Cluster centroid: %d' %i)
    aux = ''
    #finding the top 10 words for every cluster
    for j in sorted_centroids[i, :10]:
        aux += terms[j] + ' | '
        
    print(aux)

In [ ]:
print('Number of clusters:', np.unique(kmeans_clusters))
# This is the same as
print(np.unique(kmeans.labels_))

### For visualization of the K-Means Clusters, we will take a sample

In [ ]:
print('Shape of the vz_sample tfidf matrix', vz_sample.shape)

# Apply the Batched K-Means on the sample
kmeans = kmeans_model.fit(vz_sample)
# get the cluster centroids and mapping of features to the clusters
kmeans_clusters = kmeans.predict(vz_sample)
kmeans_distances = kmeans.transform(vz_sample)

# apply the T-SNE model to reduce dimension to 2, so it becomes easier to visualize
tsne_kmeans = tsne_model.fit_transform(kmeans_distances)

In [ ]:
# saving the model
pickle.dump(kmeans_model, open('tsne_kmeans_combined_sample.pkl', 'wb'))

In [ ]:
# dimension has been shrunk to 2
tsne_kmeans.shape

In [ ]:
#creata dataframe for the sample
kmeans_df = pd.DataFrame(tsne_kmeans, columns=['pc1', 'pc2'] )
kmeans_df['cluster']= kmeans_clusters
kmeans_df['description']= combined_sample['item_description']
kmeans_df['category']= combined_sample['general_cat']

kmeans_df.head()

In [ ]:
plot_kmeans = bp.figure(plot_width=700, plot_height=600,
                        title="KMeans clustering of the description",
    tools="pan,wheel_zoom,box_zoom,reset,hover,save",
    x_axis_type=None, y_axis_type=None, min_border=1)

In [ ]:
colormap = np.array(["#6d8dca", "#69de53", "#723bca", "#c3e14c", "#c84dc9", "#68af4e", "#6e6cd5",
"#e3be38", "#4e2d7c", "#5fdfa8", "#d34690", "#3f6d31", "#d44427", "#7fcdd8", "#cb4053", "#5e9981",
"#803a62", "#9b9e39", "#c88cca", "#e1c37b", "#34223b", "#bdd8a3", "#6e3326", "#cfbdce", "#d07d3c",
"#52697d", "#194196", "#d27c88", "#36422b", "#b68f79"])

In [ ]:
source = ColumnDataSource(data=dict(x=kmeans_df['pc1'], y=kmeans_df['pc2'],
                                    color=colormap[kmeans_clusters],
                                    description=kmeans_df['description'],
                                    category=kmeans_df['category'],
                                    cluster=kmeans_df['cluster']))

plot_kmeans.scatter(x='x', y='y', color='color', source=source)
hover = plot_kmeans.select(dict(type=HoverTool))
hover.tooltips={"description": "@description", "category": "@category", "cluster":"@cluster" }
show(plot_kmeans)

In [ ]:
kmeans_df

In [ ]:
# using seaborn
plt.figure(figsize=(20, 10))
sns.scatterplot(x=kmeans_df['pc1'], y=kmeans_df['pc2'], hue=kmeans_df['cluster'])

### Topic Modeling - using LDA(Latent Dirichlet Allocation)
- the input to the topic model is a bag of words, for which we will use a CountVectorizer


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cvectorizer = CountVectorizer(min_df=5, 
                             max_features=180000,
                             tokenizer = tokenize_text,
                             ngram_range = (1,2))

In [ ]:
cvz = cvectorizer.fit_transform(combined_sample['item_description'])

In [ ]:
# saving the model
pickle.dump(cvz, open('countVectorizer_for_lda.pkl', 'wb'))

In [ ]:
import logging
logging.getLogger("lda").setLevel(logging.WARNING)

from sklearn.decomposition import LatentDirichletAllocation

num_topics=20
lda_model = LatentDirichletAllocation(n_components = num_topics, 
                                     max_iter=20, 
                                     random_state= 42, 
                                     learning_method='online'
                                     )

In [ ]:
# input is a bag of words
X_topics = lda_model.fit_transform(cvz)

In [ ]:
# saving the model
pickle.dump(lda_model, open('lda_model_countVectorizer.pkl', 'wb'))

In [ ]:
print('Shape of the X_topics is:', X_topics.shape)
print(X_topics)

In [ ]:
# getting the topic components
topic_word = lda_model.components_
print('Shape of topic_word is: ', topic_word.shape)
print(topic_word)

In [ ]:
print(np.argsort(topic_word[0]))
print(np.argsort(topic_word[0])[:-10:-1])

In [ ]:
np.array(vocabulary)[np.argsort(topic_word[0])[:-10:-1]]

In [ ]:
vocabulary[1345]

In [ ]:
topic_word = lda_model.components_ # these are the 20 topics words
vocabulary = cvectorizer.get_feature_names() # bag of words

topic_summaries=[]

n_top_words =10 # 10 most relevant words related to the topic

for i, topic_dist in enumerate(topic_word):
    indexes = np.argsort(topic_dist)[:-(n_top_words+1): -1] # getting the indexes of the top 10 words, in descending order
    topic_words = np.array(vocabulary)[indexes]
    print('topic %d' %i)
    print(' | '.join(topic_words))


### X_topics:
    * each row indicates a document with a probabilistic distribution across the 20 different topics

In [ ]:
# reduce dimensions using T-SNE
print('Dimension of X_topics is :', X_topics.shape)
tsne_lda = tsne_model.fit_transform(X_topics)

In [ ]:
# saving the model
pickle.dump(tsne_lda, open('tsne_lda_model_combined_sample.pkl', 'wb'))

In [ ]:
# convert to matrix to normalize the values across columns, such that sum across rows =1
unnormalized = np.matrix(X_topics)

doc_top_normalized = unnormalized / unnormalized.sum(axis=1)
doc_top_normalized

In [ ]:
# check if normalized across rows
doc_top_normalized[0].sum()

In [ ]:
# finding the most relevant topic for each item description
lda_keys=[]
for i, description in enumerate(combined_sample['item_description']):
    lda_keys += [doc_top_normalized[i].argmax()] # get the indexes of the topic with the highest probablisitic values
    
print(lda_keys[:10])

#create a Dataframe
lda_df = pd.DataFrame(tsne_lda, columns=['pc1', 'pc2'])
lda_df['description'] = combined_sample['item_description']
lda_df['category'] = combined_sample['general_cat']
lda_df['topic'] = lda_keys
lda_df['len_docs'] = combined_sample['tokens'].map(len)


lda_df

In [ ]:
plot_lda = bp.figure(plot_width=700,
                     plot_height=600,
                     title="LDA topic visualization",
    tools="pan,wheel_zoom,box_zoom,reset,hover,save",
    x_axis_type=None, y_axis_type=None, min_border=1)

In [ ]:
source = ColumnDataSource(data=dict(x=lda_df['pc1'], y=lda_df['pc2'],
                                    color=colormap[lda_keys],
                                    description=lda_df['description'],
                                    topic=lda_df['topic'],
                                    category=lda_df['category']))

plot_lda.scatter(source=source, x='x', y='y', color='color')

hover = plot_kmeans.select(dict(type=HoverTool))
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips={"description":"@description",
                "topic":"@topic", "category":"@category"}
show(plot_lda)

In [ ]:
def prepareLDAData():
    data = {
        'vocab': vocabulary,   # vocabulary = cvectorizer.get_feature_names()
        'doc_topic_dists': doc_top_normalized,
        'doc_lengths': list(lda_df['len_docs']),
        'term_frequency':cvectorizer.vocabulary_,
        'topic_term_dists': lda_model.components_
    } 
    return data

In [ ]:
print('lda_model.components_ shape:', lda_model.components_.shape)

In [ ]:
# get the count frequency of words in the entire corpus of item description in 'combined_sample'
cvectorizer.vocabulary_

### For interactive visualization for LDA

In [ ]:
import pyLDAvis

ldadata = prepareLDAData()
pyLDAvis.enable_notebook(sort=True)
prepared_data = pyLDAvis.prepare(**ldadata)

In [ ]:
# saving the model
import pickle
with open('pyldavis_model.pkl', 'wb') as f:
    pickle.dump(prepared_data, f)

In [ ]:
pyLDAvis.save_html(prepared_data, 'prepared_data_lda.html')